# Demo Notebook for MLCommons Integration
This notebook provides a walkthrough guidance for users to invoke MLCommons apis to upload ml models to opensearch cluster

Step 0: Import packages and set up client

Step 1: Upload NLP model from local file to Opensearch cluster

Step 2: Load Model

Step 3: Get Task

Step 4: Get Model

Step 5: Generate Sentence Embedding

Step 6: Unload Model

Step 7: Delete Model



## Step 0: Import packages and set up client
Install required packages for opensearch_py_ml.sentence_transformer_model
Install `opensearchpy` and `opensearch-py-ml` through pypi


In [1]:
!pip install opensearch-py opensearch-py-ml

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings("ignore", message="Unverified HTTPS request")
from opensearchpy import OpenSearch

In [10]:
CLUSTER_URL = 'https://localhost:9200'

In [11]:
def get_os_client(cluster_url = CLUSTER_URL,
                  username='admin',
                  password='admin'):
    '''
    Get OpenSearch client
    :param cluster_url: cluster URL like https://ml-te-netwo-1s12ba42br23v-ff1736fa7db98ff2.elb.us-west-2.amazonaws.com:443
    :return: OpenSearch client
    '''
    client = OpenSearch(
        hosts=[cluster_url],
        http_auth=(username, password),
        verify_certs=False
    )
    return client 

In [12]:
client = get_os_client()

## Step 1: Upload NLP model from local file to Opensearch cluster

We can upload machine learning models to Opensearch cluster using MLCommons upload_model api. In this demo we will show how can we upload model which is stored in our local files. 

To demonstrate, we download the model zip file from the url: https://github.com/opensearch-project/ml-commons/raw/2.x/ml-algorithms/src/test/resources/org/opensearch/ml/engine/algorithms/text_embedding/all-MiniLM-L6-v2_torchscript_sentence-transformer.zip?raw=true

To upload model to the cluster, we need a zip file containing a torchScript file (.pt extension) and a tokenizer.json file. Please refer to the previous download. We also need a json file with defining the config information with following these request fields: 

https://opensearch.org/docs/latest/ml-commons-plugin/api/#request-fields


In [19]:
#connect to ml_common client with OpenSearch client
from opensearch_py_ml.ml_commons import MLCommonClient
ml_client = MLCommonClient(client)


model_path = '/Volumes/workplace/upload_content/all-MiniLM-L6-v2_torchscript_sentence-transformer.zip'
model_config_path = '/Volumes/workplace/upload_content/all-MiniLM-L6-v2_torchscript.json'

"""
all-MiniLM-L6-v2_torchscript.json content:

{
    "name": "all-MiniLM-L6-v2",
    "version": 1,
    "model_format": "TORCH_SCRIPT",
    "model_config": {
        "model_type": "bert",
        "embedding_dimension": 384,
        "framework_type": "sentence_transformers"
    }
}
"""


ml_client.upload_model(model_path, model_config_path, isVerbose=True)

Total number of chunks 9
Sha1 value of the model file:  9376c2ebd7c83f99ec2526323786c348d2382e6d86576f750c89ea544d6bbb14
Model meta data was created successfully. Model Id:  jtuRoIUBqB81FWKil3qA
uploading chunk 1 of 9
Model id: {'status': 'Uploaded'}
uploading chunk 2 of 9
Model id: {'status': 'Uploaded'}
uploading chunk 3 of 9
Model id: {'status': 'Uploaded'}
uploading chunk 4 of 9
Model id: {'status': 'Uploaded'}
uploading chunk 5 of 9
Model id: {'status': 'Uploaded'}
uploading chunk 6 of 9
Model id: {'status': 'Uploaded'}
uploading chunk 7 of 9
Model id: {'status': 'Uploaded'}
uploading chunk 8 of 9
Model id: {'status': 'Uploaded'}
uploading chunk 9 of 9
Model id: {'status': 'Uploaded'}
Model uploaded successfully


'jtuRoIUBqB81FWKil3qA'

## Step 2: Load Model

In the last step we upload a model and the model id is: `jtuRoIUBqB81FWKil3qA`. Now we will load this model in opensearch memory.

In [30]:

load_model_output = ml_client.load_model("jtuRoIUBqB81FWKil3qA")

print(load_model_output)


{'task_id': 'kNuaoIUBqB81FWKimHoo', 'status': 'CREATED'}


## Step 3: Get Task

When we invoke load model api of mlcommons plugin, a task get created. We can see the task id (`j9uRoIUBqB81FWKi_Xqu`) from previous output. Now, we can get the detailed information of the task using this task id

In [31]:

task_info = ml_client.get_task_info("kNuaoIUBqB81FWKimHoo")

print(task_info)

{'model_id': 'jtuRoIUBqB81FWKil3qA', 'task_type': 'LOAD_MODEL', 'function_name': 'TEXT_EMBEDDING', 'state': 'COMPLETED', 'worker_node': '56rNfEbPSG6p8ZZli59Zpg,Lncik04uQxe-cw3BC14wNA', 'create_time': 1673436764200, 'last_update_time': 1673436768619, 'is_async': True}


## Step 4: Get Model

With using the model id, we can also pull information about the model metadata from the opensearch cluster.

In [32]:

model_info = ml_client.get_model_info("jtuRoIUBqB81FWKil3qA")

print(model_info)

{'name': 'all-MiniLM-L6-v2', 'algorithm': 'TEXT_EMBEDDING', 'model_version': '1', 'model_format': 'TORCH_SCRIPT', 'model_state': 'LOADED', 'model_content_hash_value': '9376c2ebd7c83f99ec2526323786c348d2382e6d86576f750c89ea544d6bbb14', 'model_config': {'model_type': 'bert', 'embedding_dimension': 384, 'framework_type': 'SENTENCE_TRANSFORMERS'}, 'created_time': 1673436174206, 'last_loaded_time': 1673436768616, 'total_chunks': 9}


## Step 5: Generate Sentence Embedding

Now using the loaded model in memory, we can generate embedding for sentences. We can provide a list of sentences to get a list of embedding for the sentences. 

In [33]:
# Now using this model we can generate sentence embedding.

input_sentences = ["Test sentence1", "Test sentence2"]

embedding_output = ml_client.generate_embedding("jtuRoIUBqB81FWKil3qA", input_sentences)

print(embedding_output)


{'inference_results': [{'output': [{'name': 'sentence_embedding', 'data_type': 'FLOAT32', 'shape': [384], 'data': [0.01634426, 0.09597998, 0.019145392, 0.06294038, -0.008701664, 0.018567331, 0.10390024, 0.01438248, -0.053841658, 0.028416203, 0.12721275, -0.078425184, 0.042815633, -0.021008862, -0.0083344765, -0.015378756, 0.025418157, -0.057775266, -0.08016912, 0.04194826, 0.0363033, -0.03286257, -0.0021801298, 0.06573708, -0.0006939303, 0.016743109, -0.033662688, 0.027211811, 0.079936594, -0.033478886, -0.0053786216, 0.026796935, 0.03215637, 0.078822225, 0.058919754, -0.012447697, 0.020444121, 0.02177262, -0.01903451, 0.0064995894, 0.013272167, -0.12131438, 0.06135812, -0.029969675, 0.018355636, -0.018626912, -0.031622734, 0.050837226, -0.07211012, -0.05208043, -0.080562316, -0.03143466, -0.12672405, -0.026447112, -0.01855498, 0.008953681, -0.033323012, 0.013190011, -0.015476841, 0.02531755, 0.014650791, -0.04100695, -0.045453914, 0.013998013, 0.14769986, -0.047261648, -0.012467081, -

## Step 6: Unload Model

After generating the embedding if we want we can unload the model from memory. `unload_model` method takes two input. 

1. model_id --> Which model we want to unload
2. node_ids --> list of the nodes from where we want to unload the model.

If we don't provide `node_ids` then method will unload model from all the nodes available like the following example.

In [34]:

unload_model_response = ml_client.unload_model("jtuRoIUBqB81FWKil3qA")

print(unload_model_response)

{'Lncik04uQxe-cw3BC14wNA': {'stats': {'jtuRoIUBqB81FWKil3qA': 'unloaded'}}, '56rNfEbPSG6p8ZZli59Zpg': {'stats': {'jtuRoIUBqB81FWKil3qA': 'unloaded'}}}


## Step 7: Delete Model

We can also delete the model from the index using the model id.

In [35]:

delete_model_response = ml_client.delete_model("jtuRoIUBqB81FWKil3qA")

print(delete_model_response)

{'_index': '.plugins-ml-model', '_id': 'jtuRoIUBqB81FWKil3qA', '_version': 9, 'result': 'deleted', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 1397, '_primary_term': 38}
